<a href="https://colab.research.google.com/github/vadhri/ai-notebook/blob/main/mlops/pistachio-dataset/Experiment-Parameter-Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%%writefile train.py

import argparse
import wandb
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from fastai.vision.all import *

# Defaults for sweep overrides or direct usage
default_config = SimpleNamespace(
    loss_function="log_loss",
    learning_rate="optimal",
    num_iterations=1000,
    log_interval=100,
    wandb_project="pistachio-classification",
    wandb_run_name="sgd_classifier_run",
    use_wandb=True
)

def parse_args():
    """Override default arguments"""
    parser = argparse.ArgumentParser(description="SGD classifier training script")
    parser.add_argument("--loss_function", type=str, default=default_config.loss_function)
    parser.add_argument("--learning_rate", type=str, default=default_config.learning_rate)
    parser.add_argument("--num_iterations", type=int, default=default_config.num_iterations)
    parser.add_argument("--log_interval", type=int, default=default_config.log_interval)
    parser.add_argument("--wandb_project", type=str, default=default_config.wandb_project)
    parser.add_argument("--wandb_run_name", type=str, default=default_config.wandb_run_name)
    parser.add_argument("--use_wandb", type=bool, default=default_config.use_wandb)
    args = parser.parse_args()
    vars(default_config).update(vars(args))
    return

def load_data():
  # python funcntion to check if direectory exists
    import os
    if not os.path.exists('/content/artifacts/pistachio-image-classification-data-split:v0/data_split.csv'):
      processed_data_at = wandb.use_artifact(f'pistachio-image-classification-data-split:latest')
      processed_dataset_dir = Path(processed_data_at.download())

    data_path = '/content/artifacts/pistachio-image-classification-data-split:v0/data_split.csv'
    df = pd.read_csv(data_path)

    X = df.drop(['Class', 'Stage'], axis=1)
    y = df['Class']

    X_train = X[df['Stage'] == 'train']
    y_train = y[df['Stage'] == 'train']
    X_val = X[df['Stage'] == 'val']
    y_val = y[df['Stage'] == 'val']
    X_test = X[df['Stage'] == 'test']
    y_test = y[df['Stage'] == 'test']

    return X_train, y_train, X_val, y_val, X_test, y_test

def run_experiment(config):
    run = None
    if config.use_wandb:
        run = wandb.init(project=config.wandb_project, name=config.wandb_run_name)

    X_train, y_train, X_val, y_val, X_test, y_test = load_data()

    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_val_enc = le.transform(y_val)
    y_test_enc = le.transform(y_test)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    classes = np.unique(y_train_enc)
    class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train_enc)
    class_weight_dict = dict(zip(classes, class_weights))

    model = SGDClassifier(loss=config.loss_function, learning_rate=config.learning_rate, random_state=42, eta0=0.0001)

    results = []
    for i in range(config.num_iterations):
        model.partial_fit(X_train_scaled, y_train_enc, classes=classes)
        train_acc = accuracy_score(y_train_enc, model.predict(X_train_scaled))
        test_acc = accuracy_score(y_test_enc, model.predict(X_test_scaled))

        if i % config.log_interval == 0:
            print(f"Iteration {i+1}: Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}")

        if config.use_wandb:
            wandb.log({"iteration": i + 1, "train_accuracy": train_acc, "test_accuracy": test_acc})

        results.append({"iteration": i + 1, "train_accuracy": train_acc, "test_accuracy": test_acc})

    val_acc = accuracy_score(y_val_enc, model.predict(X_val_scaled))
    print(f"Final Validation Accuracy: {val_acc:.4f}")

    if config.use_wandb:
        wandb.log({"final_val_accuracy": val_acc})
        result_table = wandb.Table(columns=["iteration", "train_accuracy", "test_accuracy"])
        for res in results:
            result_table.add_data(res["iteration"], res["train_accuracy"], res["test_accuracy"])
        wandb.log({"training_results": result_table})
        # run.finish() # already commented out intentionally

    return {
        "model": model,
        "scaler": scaler,
        "label_encoder": le,
        "val_accuracy": val_acc,
        "training_log": results
    }

if __name__ == '__main__':
    parse_args()
    run_experiment(default_config)


Overwriting train.py


In [7]:
!python train.py --learning_rate=adaptive --loss_function=modified_huber --num_iterations=500 --wandb_project "pistachio-classification" --wandb_run_name "sgd_classifier_run"


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vadhri-venkat to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /content/wandb/run-20250413_123307-hi71ff2v
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run sgd_classifier_run
wandb: ⭐️ View project at https://wandb.ai/vadhri-venkat/pistachio-classification
wandb: 🚀 View run at https://wandb.ai/vadhri-venkat/pistachio-classification/runs/hi71ff2v
wandb: Downloading large artifact pistachio-image-classification-data-split:latest, 233.35MB. 4190 files... 
wandb:   4190 of 4190 files downloaded.  
Done. 0:0:37.9
Iteration 1: Train Acc: 0.8609, Test Acc: 0.8744
Iteration 101: Train Acc: 0.8719, Test Acc: 0.8791
Iteration 201: Train Acc: 0.8731, Test Acc: 0.8837
Iteration 301: Train Acc: 0.8737, Test Acc: 0.8837
Iteration 401: Train 

In [ ]:
%%writefile sweep.yaml
program: train.py

# Method can be grid, random or bayes
method: random

# Project this sweep is part of
project: pistachio-classification

# Metric to optimize
metric:
  name: final_val_accuracy
  goal: maximize

parameters:
  num_iterations:
    values: [1000, 1500, 2500]

  loss:
    values: ["log_loss", "hinge", "modified_huber"]

  learning_rate:
    values: ["constant", "optimal", "invscaling", "adaptive"]


In [9]:
!wandb login
!wandb sweep ./sweep.yaml

wandb: Currently logged in as: vadhri-venkat to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Creating sweep from: ./sweep.yaml
wandb: Creating sweep with ID: nieh006f
wandb: View sweep at: https://wandb.ai/vadhri-venkat/pistachio-classification/sweeps/nieh006f
wandb: Run sweep agent with: wandb agent vadhri-venkat/pistachio-classification/nieh006f


In [ ]:
!wandb agent vadhri-venkat/pistachio-classification/arii9zn3

Streaming output truncated to the last 5000 lines.
wandb:     train_accuracy ▃▁▃▅▆▆▆▇▆▇█▇▇▇▇▇▇███▇▇▇█████████████████
wandb: 
wandb: Run summary:
wandb: final_val_accuracy 0.84186
wandb:          iteration 1500
wandb:      test_accuracy 0.88372
wandb:     train_accuracy 0.87485
wandb: 
wandb: 🚀 View run sgd_classifier_run at: https://wandb.ai/vadhri-venkat/pistachio-classification/runs/bfg87oog
wandb: ⭐️ View project at: https://wandb.ai/vadhri-venkat/pistachio-classification
wandb: Synced 5 W&B file(s), 1 media file(s), 1 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20250413_134629-bfg87oog/logs
wandb: WARNING The legacy backend is deprecated. In future versions, `wandb-core` will become the sole backend service, and the `wandb.require('legacy-service')` flag will be removed. For more information, visit https://wandb.me/wandb-core
2025-04-13 13:46:44,800 - wandb.wandb_agent - INFO - Cleaning up finished run: bfg87oog
2025-04-13 13:46:45,004 - wandb.wandb_agent